In [1]:
import awpy

In [2]:
#path = '/content/drive/MyDrive/data/lan/'
path = '/Users/hemanth/Downloads/esta-main/data/lan/'

In [3]:
import lzma
import json

# Function to read .xz archives from ESTA
def read_parsed_demo(filename):
  with lzma.LZMAFile(filename, "rb") as f:
    d = json.load(f)
    return d

In [6]:
demoJson = read_parsed_demo(path + '0013db25-4444-452b-980b-7702dc6fb810.json.xz');

In [7]:
demoJson.keys()

dict_keys(['clientName', 'mapName', 'tickRate', 'playbackTicks', 'playbackFramesCount', 'parsedToFrameIdx', 'parserParameters', 'serverVars', 'matchPhases', 'matchmakingRanks', 'playerConnections', 'gameRounds', 'matchId', 'demoId', 'competitionName', 'hltvUrl', 'matchDate', 'matchName'])

In [16]:
# Function to create a vector representation for a game state (also called a frame)
def create_frame_row(frame, demoID, mapName, roundNum):

  # Global info
  frame_data = [demoID, mapName, roundNum, frame["seconds"]]
    
  # Team specific info (CT)
  for p in frame["ct"]["players"]:
    player_info = [p["isAlive"], 
                  p["x"],
                  p["y"],
                  p["z"],
                  p["velocityX"],
                  p["velocityY"],
                  p["velocityZ"],
                  p["viewX"],
                  p["viewY"]
                  ]
    frame_data.extend(player_info)
  
  for p in frame["t"]["players"]:
    player_info = [p["isAlive"], 
                  p["x"],
                  p["y"],
                  p["z"],
                  p["velocityX"],
                  p["velocityY"],
                  p["velocityZ"],
                  p["viewX"],
                  p["viewY"]
                  ]
    frame_data.extend(player_info)
  
  return frame_data
    

In [17]:
import os
import pandas as pd

from tqdm import tqdm

demo_files = os.listdir(path)
demo_files = [path + f for f in demo_files] 
parsed_demos = []

# You can change the list of maps here to only parse specific maps
map_list = ['de_nuke', 'de_inferno', 'de_vertigo', 'de_dust2', 'de_mirage', 'de_overpass', 'de_ancient']

for f in tqdm(demo_files):
  demo = read_parsed_demo(f)

  if demo["mapName"] in map_list:
    # parsed_demos[demo["mapName"]][demo["demoId"]] = {}
    # parsed_demos[demo["mapName"]][demo["demoId"]]["df"] = []

    parsed_frames_df = []

    for r in demo["gameRounds"]:
      parsed_frames_df_round = []

      ct_win = 1

      # Only parse rounds where the round end reason is one of the following. These rounds are 99.9% of rounds in ESTA. 
      if r["roundEndReason"] in ["CTWin", "TargetSaved", "BombDefused", "TargetBombed", "TerroristsWin"]:
        if r["roundEndReason"] not in ["CTWin", "TargetSaved", "BombDefused"]:
          ct_win = 0
        for fr in r["frames"]:
          # We use the following filters to ensure that we do not get frames after the round has ended
          if (fr["ct"]["players"] is not None) & (fr["t"]["players"] is not None) & (fr["clockTime"] != "00:00") & (fr["t"]["alivePlayers"] >= 0) & (fr["ct"]["alivePlayers"] >= 1):
            if (len(fr["ct"]["players"]) == 5) & (len(fr["t"]["players"]) == 5):

              # Create dataframe/tabular representation
              frame_row = create_frame_row(fr, demo["demoId"], demo["mapName"], r["roundNum"])
              frame_row.append(ct_win)
              
              parsed_frames_df_round.append(frame_row)

        if len(parsed_frames_df_round) > 0:
          parsed_demos.extend(parsed_frames_df_round)


00%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 680/680 [1:40:17<00:00,  8.85s/it]

In [18]:
import pandas as pd

lan_frames_df = pd.DataFrame(parsed_demos)

In [15]:
lan_frames_df[94].value_counts()

94
0    2653
1    2119
Name: count, dtype: int64

In [19]:
lan_frames_df.to_pickle('lan_frames_df.pkl')

In [ ]:
#lan_frames_df = pd.read_pickle('lan_frames_df.pkl')

In [23]:
lan_frames_df.shape

(3338527, 95)

In [20]:
lan_frames_df.head()

,0,1,2,3,4,5,6,7,8,9,...,85,86,87,88,89,90,91,92,93,94
0,cc25ba4e-66d1-46b0-99c6-b288986547ca,de_mirage,1,0.484375,True,-1699.797729,-1970.743164,-268.632446,29.026894,-227.075592,...,True,1350.458008,-89.996193,-167.968750,-136.127853,-178.551804,0.000000,255.481567,2.147827,0
1,cc25ba4e-66d1-46b0-99c6-b288986547ca,de_mirage,1,0.984375,True,-1778.514404,-1802.967896,-263.968750,220.579300,117.663780,...,True,1207.346436,-347.120575,-159.529190,57.878857,-243.207825,280.118378,288.956909,14.826050,0
2,cc25ba4e-66d1-46b0-99c6-b288986547ca,de_mirage,1,1.484375,True,-1605.518311,-2181.437256,-255.857178,198.653351,-151.778946,...,True,1165.320923,-535.673462,-192.924774,-44.098000,-246.080002,0.000000,259.804688,15.380859,0
3,cc25ba4e-66d1-46b0-99c6-b288986547ca,de_mirage,1,1.984375,True,-1597.993774,-1278.946289,-261.710175,-4.611290,249.957474,...,True,1357.354004,-662.884827,-167.968750,-11.778632,-249.722366,0.000000,266.967773,0.461426,0
4,cc25ba4e-66d1-46b0-99c6-b288986547ca,de_mirage,1,2.484375,True,-1570.984253,-1494.990601,-263.379761,24.567797,248.789871,...,True,1166.454346,-715.387939,-236.118439,-22.092203,-249.021957,0.000000,264.979248,11.700439,0


In [25]:
test = lan_frames_df[lan_frames_df[0]=='cc25ba4e-66d1-46b0-99c6-b288986547ca']

In [28]:
test = test[test[2]==1]

In [29]:
test.shape

(141, 95)

In [27]:
test[2].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])